In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Back Translation

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
!pip install googletrans==4.0.0-rc1 kss

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 52.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 51.1 MB/s eta 0:00:00

In [ ]:
import json
import kss
from googletrans import Translator
import time
from tqdm import tqdm
import os

input_path = "/content/drive/MyDrive/ABSA/data/base_data.jsonl"
output_path = "/content/drive/MyDrive/ABSA/data/aug_en_withko.jsonl"
target_languages = ['en']
delay = 1.5
checkpoint_interval = 25

def back_translate(text, translator, src='ko', mid='en', delay=1.5):
    sentences = kss.split_sentences(text)
    result = []
    for sent in sentences:
        try:
            mid_sent = translator.translate(sent, src=src, dest=mid).text
            time.sleep(delay)
            back_sent = translator.translate(mid_sent, src=mid, dest=src).text
            time.sleep(delay)
            result.append(back_sent)
        except Exception as e:
            print(f"[{mid}] 번역 실패: {e}")
            result.append(sent)
    return ' '.join(result)

augmented = []
translated_cache = set()

if os.path.exists(output_path):
    with open(output_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            augmented.append(entry)
            translated_cache.add((entry['text'], entry['origin']))

translator = Translator()

with open(input_path, 'r', encoding='utf-8') as f:
    base_data = [json.loads(line) for line in f]

for i, item in enumerate(tqdm(base_data, desc="역번역 진행 중", total=len(base_data))):
    text = item['text']
    entities = item['label']

    if (text, 'ko') not in translated_cache:
        augmented.append({
            "text": text,
            "entities": entities,
            "origin": "ko"
        })
        translated_cache.add((text, 'ko'))

    for lang in target_languages:
        if (text, lang) in translated_cache:
            continue

        back_text = back_translate(text, translator, src='ko', mid=lang, delay=delay)
        augmented.append({
            "text": back_text,
            "entities": entities,
            "origin": lang
        })
        translated_cache.add((text, lang))

    if i % checkpoint_interval == 0 and i > 0:
        with open(output_path, 'w', encoding='utf-8') as f:
            for entry in augmented:
                f.write(json.dumps(entry, ensure_ascii=False) + '\n')
        print(f"체크포인트 저장 완료: {i}개 문장 처리")

with open(output_path, 'w', encoding='utf-8') as f:
    for entry in augmented:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"최종 저장 완료: {output_path}")

역번역 진행 중:   2%|▏         | 26/1569 [05:36<4:51:30, 11.34s/it]

체크포인트 저장 완료: 25개 문장 처리


역번역 진행 중:   3%|▎         | 51/1569 [11:33<5:54:33, 14.01s/it]

체크포인트 저장 완료: 50개 문장 처리


역번역 진행 중:   5%|▍         | 76/1569 [16:51<5:24:24, 13.04s/it]

...

[en] 번역 실패: The read operation timed out


역번역 진행 중:  94%|█████████▍| 1476/1569 [8:49:03<40:21, 26.03s/it]

체크포인트 저장 완료: 1475개 문장 처리


역번역 진행 중:  95%|█████████▍| 1483/1569 [8:52:29<30:52, 21.54s/it]


KeyboardInterrupt: 

In [ ]:
import json

input_path = "/content/drive/MyDrive/ABSA/data/aug_en_withko.jsonl"  # 원본 파일 경로
output_path = "/content/drive/MyDrive/ABSA/data/aug_en.jsonl"  # 결과 저장 경로

even_lines = []

with open(input_path, 'r', encoding='utf-8') as f:
    for idx, line in enumerate(f):
        if idx % 2 == 0:
            data = json.loads(line)
            even_lines.append(data)

with open(output_path, 'w', encoding='utf-8') as f:
    for item in even_lines:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"짝수 번째 행 {len(even_lines)}개가 {output_path}에 저장되었습니다.")

In [ ]:
import json

input_path = "/content/drive/MyDrive/ABSA/data/aug_en.jsonl"
output_path = "/content/drive/MyDrive/ABSA/data/aug_en_nn.jsonl"

filtered_data = []

with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        filtered_entities = [
            entity for entity in entry.get("label", [])
            if entity[2].endswith("부정") or entity[2].endswith("중립")
        ]
        if filtered_entities:
            filtered_data.append({
                "text": entry["text"],
                "entities": filtered_entities,
                "origin": entry.get("origin", "")
            })

with open(output_path, 'w', encoding='utf-8') as f:
    for entry in filtered_data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"완료: {len(filtered_data)}개 문장이 {output_path}에 저장되었습니다.")